In [1]:
!pip install opencv-python mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 39.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 18.1.0 which is inc

In [26]:
import mediapipe as mp
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# Initialize drawing utilities for visualization (optional)
mp_drawing = mp.solutions.drawing_utils


W0000 00:00:1733923961.788905     187 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733923961.842983     187 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [27]:
def calculate_posture_score(landmarks, frame_shape):
    """
    Calculate posture score based on shoulder alignment and spine angle.
    Score ranges from 0 (slouched) to 10 (upright).
    """
    image_height, image_width, _ = frame_shape
    # Extract required landmarks
    left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_hip = landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
    nose = landmarks.landmark[mp_pose.PoseLandmark.NOSE]

    # Convert normalized coordinates to pixel values
    def to_pixel(landmark):
        return np.array([landmark.x * image_width, landmark.y * image_height])

    left_shoulder_px = to_pixel(left_shoulder)
    right_shoulder_px = to_pixel(right_shoulder)
    left_hip_px = to_pixel(left_hip)
    right_hip_px = to_pixel(right_hip)
    nose_px = to_pixel(nose)

    # 1. Shoulder Alignment
    shoulder_diff = abs(left_shoulder_px[1] - right_shoulder_px[1])
    max_shoulder_diff = 50  # pixels, adjust based on expected maximum
    shoulder_alignment_score = max(0, 10 - (shoulder_diff / max_shoulder_diff) * 10)
    shoulder_alignment_score = min(shoulder_alignment_score, 10)

    # # 2. Spine Angle
    # # Calculate midpoint of shoulders and hips
    shoulder_mid = (left_shoulder_px + right_shoulder_px) / 2
    hip_mid = (left_hip_px + right_hip_px) / 2

    # # Vector from hip to shoulder
    spine_vector = shoulder_mid - hip_mid
    spine_length = np.linalg.norm(spine_vector)
    

    # Calculate angle with vertical
    vertical_vector = np.array([0, -1])  # Upwards
    if spine_length != 0:
        spine_unit = spine_vector / spine_length
        dot_product = np.dot(spine_unit, vertical_vector)
        angle_rad = math.acos(np.clip(dot_product, -1.0, 1.0))
        angle_deg = math.degrees(angle_rad)

    # Assuming 0 degrees is perfect vertical, and up to 30 degrees slouch
    max_angle = 30  # degrees
    spine_angle_score = max(0, 10 - (angle_deg / max_angle) * 10)
    spine_angle_score = min(spine_angle_score, 10)

    # 3. Head Position (optional)
    # Ensure head is aligned over shoulders
    head_diff = abs(nose_px[1] - shoulder_mid[1])
    max_head_diff = 300  # pixels, adjust as needed
    head_position_score = max(0, 10 - (head_diff / max_head_diff) * 10)
    head_position_score = min(head_position_score, 10)

    # Combine the scores with weights
    # Adjust weights as per importance
    total_score = (shoulder_alignment_score * 0.4 +
                  spine_angle_score * 0.5 +
                  head_position_score * 0.1)
    print("shoulder",shoulder_alignment_score)
    print('spine',spine_angle_score)
    print('head',head_position_score)
    # Normalize to 0-10
    total_score = max(0, min(total_score, 10))

    return round(total_score, 2),round(shoulder_alignment_score, 2),round(spine_angle_score, 2),round(head_position_score, 2)


In [29]:
try:
    print(final_posture_score/count)
    print(final_shoulder_score/count)
    print(final_spine_score/count)
    print(final_head_score/count)
except:
    print("give proper video")

5.682187499999997
3.3708124999999987
7.696687499999999
4.852500000000002


In [30]:
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh()
# results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
